# Learning Eikon Data API - Internship Diary (Part 1)

## Introduction

My name is Warisara Chewpraditkul (Praw) currently studying in Computer Engineering at [King Mongkut's University of Technology Thonburi](https://global.kmutt.ac.th/). I had an opportunity to participate in an internship program at the Refinitiv Software (Thailand) for two months, from June to July 2020.  During this period, I started learning [Eikon Data API](https://developers.refinitiv.com/eikon-apis/eikon-data-api) and then used it to develop Python applications in Notebook to retrieve and display financial information, such as news, historical, and reference data. In this article, I would like to share my Eikon Data API's knowledge and experience with all of you.

## Eikon Data API

First, I will briefly introduce Eikon Data API. Eikon Data API provides interfaces for developers to access data directly from Eikon Desktop or Refinitiv Workspace. It leverages in-house or third-party desktop applications and provides a seamless workflow with the same data across all applications running on the desktop. The data that Eikon Data API can provide includes real-time streaming, historical, news, symbology, and reference data. 

Before using Eikon Data API, you must have the following things.

1. **Refinitiv Workspace or Eikon Desktop:** Eikon Data API connects to Refinitiv Workspace or Eikon Desktop to retrieve the data. For this reason, the Refinitiv Workspace or Eikon Desktop must be running on the same machine where you use the Eikon Data API. You can download Eikon Desktop from [https://eikon.refinitiv.com](https://eikon.refinitiv.com),  or Refinitiv Workspace from  [https://workspace.refinitiv.com](https://workspace.refinitiv.com). 
2. **Login Credentials:** To use Eikon Desktop or Refinitiv Workspace, the login credentials (username and password) are required and the credentials must have API feature enabled. Please contact Refinitiv sales or account team to obtain this information.
3. **App Key:** Every application using the Eikon Data API must identify itself with App Key. The App Key can be created via the App Key Generator application inside Eikon Desktop or Refinitiv Workspace. Please refer to [Step 2. Create an App Key for your application in Eikon Data API Quick Start](https://developers.refinitiv.com/eikon-apis/eikon-data-api/quick-start#create-app-key) for more information.
4. **Eikon Data API Libraries:** At the time of this writing, Eikon Data API libraries are available in [Python](https://pypi.org/project/eikon/) and [.NET](https://www.nuget.org/packages/EikonDataAPI/). The Eikon Data API Python library is officially supported by Refinitiv while the Eikon Data API .NET library is supported by the Refinitiv developers' community. Please refer to [Step 3. Install the Python library for the Eikon Data API in Eikon Data API Quick Start](https://developers.refinitiv.com/eikon-apis/eikon-data-api/quick-start#install-python-library) and [.NET Libraries for Eikon Data API Quick Start](https://developers.refinitiv.com/article/net-libraries-eikon-data-apis-quick-start) to install the Eikon Data API Python and .NET libraries, respectively.

In the following sections, I use the Eikon Data API Python library with Python 3.7.6.

In [ ]:
import sys 
print(sys.version) #python version
import warnings
warnings.filterwarnings('ignore') #prevent warning message 

Next, I will install the Eikon Data API Python library by using the pip command:

In [ ]:
pip install eikon

Then, import the **Eikon Data API** package

In [ ]:
import eikon as ek 
ek.__version__ #eikon version

To connect to **Eikon Data API** you have to run this code:

In [ ]:
#connecting to Eikon Data API
ek.set_app_key('[APP_KEY]')

**Note:**

   - Please change <APP_KEY> to your App Key. To create an App Key, please refer to [Create an App Key for your application](https://developers.refinitiv.com/eikon-apis/eikon-data-api/quick-start#create-app-key).
   - Before running the application, please make sure that **Refinitiv Workspace** or **Eikon Desktop** is running on the machine.

## Funtionalities

There are several functions available in Eikon Data API used to retrieve different kinds of data:

* **ek.get_symbology()**: This function is used to convert instrument names into other instrument codes
* **ek.get_data()**: This function is used to retrieve reference data and snapshot of real-time data
* **ek.get_timeseries()**: This function is used to retrieve inter-day and intra-day historical data
* **ek.get_news_headlines()**: This function is used to retrieve news headlines
* **ek.get_news_story()**: This function is used to retrieve full news story texts

If you need more detail about how to use each function, you can use the Python **help()**function to display the documentation of modules, classes, functions, and variables in the Python packages, or you can access the Eikon Data API online document at [Eikon Data APIs for Python - Reference Guide](https://docs-developers.refinitiv.com/1594089279984/14684/book/en/index.html).


## Symbology
The first function is `ek.get_symbology()` that will return a list of instrument names converted into another instrument codes. For example, it can convert ISIN instrument names to RIC names. I admitted that at first, I don't know what RIC and ISIN are. However, after asking my supervisor and doing some research, those are standards used to identify financial instruments. For instance:
- `RIC` stands for Refinitiv Identifier Code, which is a ticker-like code with the exchange code used by Refinitiv to identify financial instruments, such as AAPL.O.
- `ISIN` stands for International Securities Identification Numbering system used to uniquely identify securities including stocks, bonds, options, and futures, such as US0378331005.

RICs are used thoroughly in Refinitiv's products. You can search for RICs by using the [RIC Search tool](https://developers.refinitiv.com/elektron/elektron-sdk-cc/dev-tools?type=ric). 

However, to programmatically convert RICs to other codes or vice versa, the `get_symbology()` method can be used. 

Next, I'll show you how `ek.get_symbology()` works.

If you have a list of RICcodes, you can call the `ek.get_symbology()` method to convert `RIC` to `ISIN`.

In [ ]:
ric = ['AAPL.O', 'AMZN.O', 'MSFT.O', 'PTTEP.BK']
isin = ek.get_symbology(ric, from_symbol_type='RIC', to_symbol_type= 'ISIN')
isin

In additional to **RIC** and **ISIN**, there are other codes that are used globally such as **ticker**,  **SEDOL** (Stock Exchange Daily Official List), **CUSIP** (Committee on Uniform Securities Identification Procedures), etc.

With the **ek.get_symbology** method, I can convert one instrument code to many codes. For example, I can convert **RIC** to **ticker**, **SEDOL**, **CUSIP**, and **ISIN**.

In [ ]:
ek.get_symbology(ric, from_symbol_type='RIC', to_symbol_type=['ticker', 'CUSIP','SEDOL','ISIN'])

There is no guarantee that a conversion is possible. From the output above, you will see that there are some fields in the table that have no value (NaN) which means they are not available in those standards. 

## Reference Data
The second function is `ek.get_data()`, a general-purpose data retrieval function that returns pandas.DataFrame with fields in columns. For example, the code below retrieves the snapshot of real-time data and reference data, such as the latest closing price, the low price for the latest trading day, the total return, and the gross profit of IBM.N and MSFT.O.

In [ ]:
data, err = ek.get_data(['IBM.N','MSFT.O'], ['TR.PriceClose', 'TR.PriceLow', 'TR.TotalReturn1Mo',
                                    'TR.TotalReturn3Mo','TR.TotalReturn6Mo','TR.TotalReturnYTD','TR.GrossProfit(Period=FY0)'])
data

**Tip:** You can use the **Data Item Browser** to discover fields and parameters, and then copy field names and parameters into your code. In the example below, I used **Data Item Browser** to find fields and parameters to show the gross profit based on the last reported year: **'TR.GrossProfit(Period=FY0)'**.


After getting the data from the `ek.get_data()` method, you can use it to plot a graph, as shown below, to visualize and compare the data.

First, I have to install the Cufflinks library used to create graphs directly from Pandas. 

In [ ]:
pip install cufflinks

Then, import the **Cufflinks** package

In [ ]:
#import cufflinks to create the graph
import cufflinks as cf 
cf.set_config_file(offline=True) #set the plotting mode to offline
cf.__version__ #cufflinks version

In [ ]:
data.set_index('Instrument')[['1 Month Total Return','3 Month Total Return',
                              '6 Month Total Return','YTD Total Return']].iplot(kind='bar')

The above code plots a bar chart that displays the total return values of IBM.N and MSFT.O. 

## Time-series Data
The third function is `ek.get_timeseries()` used to retrieve inter-day and intra-day historical data. I'll demonstrate how to use this function below.

In [ ]:
rics = ['AAPL.O',  # Apple stock
        'AMZN.O',  # Amazon stock
       ]

The following code retrieves **daily closing prices** from 1 Jan 2017 to 23 Feb 2018 for the above list of RICs.

In [ ]:
ts_data = ek.get_timeseries( rics,
                             start_date='2017-01-01',
                             end_date='2018-02-23',
                             fields='CLOSE',
                             interval='daily'
                            )
ts_data.head()

Then, I use the output from `ek.get_timeseries()` to plot a line chart to visualize the trend of the stock prices for each company.

In [ ]:
ts_data.iplot()

The line chart above shows the real stock prices of these two companies. However, to compare the performances between the two companies, you can use `.normalize()` to normalize financial time series data and then plot the line chart to visualize the comparison.

In [ ]:
ts_data.normalize().iplot()

From the above chart, I used Amazon and Apple stock prices starting from  01 Jan 2017 as a baseline. You can see that the performances of both companies were nearly equivalent in 2017. However, at the beginning of 2018, Amazon outperformed Apple significantly.

## News Headlines
The next function is `ek.get_news_headlines()` used to search and retrieve news headlines. The output of this method includes the news headlines and storyId values used to retrieve the full news stories text. The following code retrieves 10 English news headlines starting from 1 Jan 2020 that relate to COVID and Apple Inc. 

**Tip:** Use 'R:' in front of RIC names to improve performance.

In [ ]:
from datetime import datetime #classes for manipulating dates and times
news_aapl = ek.get_news_headlines("R:AAPL.O AND Topic:COVID IN ENGLISH", date_from='2020-01-01',
                         date_to=datetime.now(), count=10)
news_aapl

The output is a data frame indexed by the news' created time. The news headlines are available in the text column. 

## News Stories
The last function is `ek.get_news_story()` used to retrieve the full text of news story for the given storyId. The storyId values are available in the storyId column returned by the `ek.get_news_headlines()` method.

In [ ]:
news_aapl['storyId']

The code below uses the storyId in the third row to retrieve the news story text. 

In [ ]:
storyId_aapl = news_aapl['storyId'][2] 
ek.get_news_story(storyId_aapl)

As you can see from the above result, the output is in HTML format. To display it nicely, you can use the Ipython API to render it.

In [ ]:
from IPython.display import HTML #rendered HTML output
HTML(ek.get_news_story(storyId_aapl))

## Summary

At this point, I learn about Eikon Data API step by step and go through each function of Eikon Data API to verify how it works, how the output of each function looks, and how those data can be used. 


## What’s Next?
For the next step, I'll use some functions in Eikon Data API to create interactive widgets:
[Learning Eikon Data API (Part 2)](https://developers.refinitiv.com/article/learning-eikon-data-api-internship-diarypart2)

## Learn more

To learn more about the **Eikon Data API** Libraries, you can join the [Refinitiv Developer Community](https://developers.refinitiv.com/) by registering and login to the Refinitiv Developer Community portal.  You will get free access to a number of learning materials, such as [Quick Start guides](https://developers.refinitiv.com/eikon-apis/eikon-data-api/quick-start), [Tutorials](https://developers.refinitiv.com/eikon-data-apis/learning), [Documentation](https://developers.refinitiv.com/eikon-apis/eikon-data-api/docs), and much more.

## Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Eikon Data APIs Q&A Forum](https://developers.refinitiv.com/eikon-apis/eikon-data-api/qa). The Refinitiv Developer Community will be very pleased to help you.

## References


1. CHEN, J., 2019. International Securities Identification Number (ISIN). [online] Investopedia. Available at: <https://www.investopedia.com/terms/i/isin.asp> [Accessed 10 August 2020].
2. CHEN, J., 2019. Stock Exchange Daily Official List (SEDOL) Definition. [online] Investopedia. Available at: <https://www.investopedia.com/terms/s/sedol.asp> [Accessed 10 August 2020].
3. Developers.refinitiv.com. n.d. Eikon Data API | DEVELOPER COMMUNITY. [online] Available at: <https://developers.refinitiv.com/eikon-apis/eikon-data-api> [Accessed 10 August 2020].
4. Jintamalit, C., 2019. How To Discover Available Fields For Data Grid Service On JET(App Studio HTML5 SDK) API Using Eikon Desktop | DEVELOPER COMMUNITY. [online] Developers.refinitiv.com. Available at: <https://developers.refinitiv.com/article/how-discover-available-fields-d... [Accessed 10 August 2020].
5. Sec.gov. 2015. SEC.Gov | CUSIP Number. [online] Available at: <https://www.sec.gov/answers/cusip.htm> [Accessed 10 August 2020].